In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from superRes.generators import *
from superRes.critics import *
from superRes.dataset import *
from superRes.loss import *
from superRes.save import *
from superRes.fid_loss import *
from superRes.ssim import *
from superRes.metrics import *

import torchvision
import geffnet # efficient/ mobile net
from PIL import Image as PilImage
import cv2

In [2]:
import time

In [3]:
path = Path('./dataset/')

path_fullRes = path/'DIV2K_train_HR'

path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'
path_lowRes_Full = path/'DIV2K_train_LR_Full_QF20'

In [4]:
models.resnet101

<function torchvision.models.resnet.resnet101(pretrained=False, progress=True, **kwargs)>

In [4]:
model = geffnet.mobilenetv3_rw
# model = models.resnet101
loss_func = lpips_loss()

bs=1
# size=(1024, 1130)

data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=1024)

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

Setting up Perceptual loss...
Loading model from: /data/students_home/fmameli/anaconda3/envs/fastai/lib/python3.6/site-packages/perceptual_similarity/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done


In [5]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

Learner(data=ImageDataBunch;

Train: LabelList (800 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_Full_QF20;

Valid: LabelList (100 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_Full_QF20;

Test: None, model=DynamicUnetWide(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): HardSwish()
      (3): Sequential(
        (0): Sequential

In [6]:
learn_gen.summary()

DynamicUnetWide
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [16, 512, 512]       432        False     
______________________________________________________________________
BatchNorm2d          [16, 512, 512]       32         True      
______________________________________________________________________
HardSwish            [16, 512, 512]       0          False     
______________________________________________________________________
Conv2d               [16, 512, 512]       144        False     
______________________________________________________________________
BatchNorm2d          [16, 512, 512]       32         True      
______________________________________________________________________
ReLU                 [16, 512, 512]       0          False     
______________________________________________________________________
Identity             [16, 512, 512]       0          False     
______________________________________________

In [66]:
data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=1, sz=512)
learn_gen.data = data_gen

In [7]:
learn_gen.export("/data/students_home/fmameli/repos/SuperRes/models/export.pkl" )

In [8]:
learn = load_learner("/data/students_home/fmameli/repos/SuperRes/models/")

In [10]:
i = 803
id_img = str(i).zfill(4)

img_low = open_image("dataset/DIV2K_train_LR_512_QF20/" + id_img + ".jpg")
    
p,img_hr,b = learn.predict(img_low)

In [12]:
start_time = time.time()
p,img_hr,b = learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6109883785247803 seconds ---


In [15]:
start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
learn.predict(img_low)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6027147769927979 seconds ---
--- 0.5905919075012207 seconds ---
--- 0.6415431499481201 seconds ---
--- 0.6389188766479492 seconds ---
--- 0.6458814144134521 seconds ---
--- 0.6176459789276123 seconds ---
--- 0.6169188022613525 seconds ---


In [ ]:
learn_gen.summary()

In [17]:
1.68/4

0.42

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_LPIPS_Tuned_gen_512px_0"
learn_gen.load(weights, with_opt=False)

i = 803
id_img = str(i).zfill(4)

img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
size=toEven(img_low.size)
data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

learn_gen.data = data_gen
    
p,img_hr,b = learn_gen.predict(img_low)
# p.save("output_imgs/" + id_img + "_PATCH.png")
# print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
p

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

for i in range(649, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_GAN.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_MSE_gen_512px_2"
learn_gen.load(weights, with_opt=False)

for i in range(649, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_MSE.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_SSIM_gen_512px_0"
learn_gen.load(weights, with_opt=False)

for i in range(1, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_SSIM.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
def toEven(sz):
    tempSz = [sz[0], sz[1]]
    if sz[0]%2 != 0:
        tempSz[0] += 1
    if sz[1]%2 != 0:
        tempSz[1] += 1
    return tempSz

In [ ]:
id_img = "0165"

img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
size=toEven(img_low.size)
data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

learn_gen.data = data_gen

In [ ]:
img_low

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

p_gan,img_hr,b = learn_gen.predict(img_low)
p_gan

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_MSE_gen_512px_2"
learn_gen.load(weights, with_opt=False)

p_mse,img_hr,b = learn_gen.predict(img_low)
p_mse

In [ ]:
p_gan.save("output_imgs/" + id_img + "_GAN.png")
p_mse.save("output_imgs/" + id_img + "_MSE.png")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_SSIM_gen_512px_0"
learn_gen.load(weights, with_opt=False)

p,img_hr,b = learn_gen.predict(img_low)
p

In [ ]:
p.save("output_imgs/" + id_img + "_SSIM.png")

In [ ]:
img_low

In [ ]:
img_low.save('output_imgs/002_1k_original.png')

# Filter based

In [ ]:
class BaseFilter():
    def __init__(self, learn: Learner, map_to_orig: bool = False):
        self.learn = learn
        self.norm, self.denorm = normalize_funcs(*imagenet_stats)
        self.render_base = 16
        self.map_to_orig = map_to_orig


    def _scale_to_square(self, orig: PilImage, targ: int) -> PilImage:
        # a simple stretch to fit a square really makes a big difference in rendering quality/consistency.
        # I've tried padding to the square as well (reflect, symetric, constant, etc).  Not as good!
        targ_sz = (targ, targ) 
        return orig.resize(targ_sz, resample=PIL.Image.BILINEAR)

    def _get_model_ready_image(self, orig: PilImage, sz: int) -> PilImage:
        result = self._scale_to_square(orig, sz)
        return result

    def _model_process(self, orig: PilImage, sz: int) -> PilImage:
        model_image = self._get_model_ready_image(orig, sz)
        x = pil2tensor(model_image, np.float32)
        x.div_(255)
        x, y = self.norm((x, x), do_x=True)
        result = self.learn.pred_batch(
            ds_type=DatasetType.Valid, batch=(x[None].cuda(), y[None]), reconstruct=True
        )
        out = result[0]
        out = self.denorm(out.px, do_x=False)
        out = image2np(out * 255).astype(np.uint8)
        return PilImage.fromarray(out)

    def _unsquare(self, image: PilImage, orig: PilImage) -> PilImage:
        targ_sz = orig.size
        image = image.resize(targ_sz, resample=PIL.Image.BILINEAR)
        return image
    
    def _post_process(self, raw_color: PilImage, orig: PilImage) -> PilImage:
        raw_color = self._unsquare(raw_color, orig)
        color_np = np.asarray(raw_color)
        orig_np = np.asarray(orig)
        color_yuv = cv2.cvtColor(color_np, cv2.COLOR_BGR2YUV)
        # do a black and white transform first to get better luminance values
        orig_yuv = cv2.cvtColor(orig_np, cv2.COLOR_BGR2YUV)
        hires = np.copy(orig_yuv)
        hires[:, :, 1:3] = color_yuv[:, :, 1:3]
        final = cv2.cvtColor(hires, cv2.COLOR_YUV2BGR)
        final = PilImage.fromarray(color_np)
        return final
    
    def filter(
        self, orig_image: PilImage, filtered_image: PilImage, render_factor: int
    ) -> PilImage:
        render_sz = orig_image.size[0] * render_factor
        print(render_sz)
        model_image = self._model_process(orig=filtered_image, sz=render_sz)

        if self.map_to_orig:
            return self._post_process(model_image, orig_image)
        else:
            return self._post_process(model_image, filtered_image)

In [ ]:
def get_dummy_databunch() -> ImageDataBunch:
    path = Path('./dummy/')
    return get_colorize_data(
        sz=1, bs=1, crappy_path=path, good_path=path, keep_pct=0.001
)

In [ ]:
bf = BaseFilter(learn_gen)

In [ ]:
img_low

In [ ]:
pred_img = bf.filter(img_low, img_low, 1); pred_img

In [ ]:
pred_img